In [1]:
from dask.distributed import Client
import numpy as np
import dask.array as da
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split
from time import time

In [2]:
client = Client(n_workers=4, threads_per_worker=2, 
                processes=False, 
                memory_limit='4 GB',
                silence_logs='error')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.42.0.224:8787/status,
Dashboard: http://10.42.0.224:8787/status,Workers: 4
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: False
Comm: inproc://10.42.0.224/21028/1,Workers: 4
Dashboard: http://10.42.0.224:8787/status,Total threads: 8
Started: Just now,Total memory: 14.90 GiB
Comm: inproc://10.42.0.224/21028/5,Total threads: 2
Dashboard: http://10.42.0.224:50983/status,Memory: 3.73 GiB
Nanny: None,


In [3]:
N_samples = 12000000
N_features = 500

In [4]:
from dask_ml.datasets import make_classification

In [5]:
X, y = make_classification(
    n_samples=N_samples,n_features=N_features, 
    n_redundant=0, n_informative=N_features-5,n_classes=2,
    flip_y=0.05,class_sep=0.8,
    chunks=10000)

c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\dask_ml\datasets.py:377: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  z0 = X[:, informative_idx].dot(beta[informative_idx])


In [6]:
X = X.astype(np.float16)
y = y.astype(np.float16)

In [7]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=101)

In [8]:
X_train.shape

(9600000, 500)

In [9]:
type(X_train)

dask.array.core.Array

In [10]:
X_train

dask.array<concatenate, shape=(9600000, 500), dtype=float16, chunksize=(8000, 500), chunktype=numpy.ndarray>

In [11]:
X_test

dask.array<concatenate, shape=(2400000, 500), dtype=float16, chunksize=(2000, 500), chunktype=numpy.ndarray>

In [12]:
X_train[:5,:5]

dask.array<getitem, shape=(5, 5), dtype=float16, chunksize=(5, 5), chunktype=numpy.ndarray>

In [13]:
X_train[:5,:5].compute()

array([[-0.841 ,  0.6387,  1.411 ,  1.13  ,  0.7036],
       [-1.997 , -0.2998, -1.812 ,  0.2979,  2.398 ],
       [ 0.748 , -2.098 , -0.793 , -0.296 , -0.1456],
       [ 1.428 , -1.344 ,  1.741 , -0.81  ,  1.796 ],
       [-1.563 , -0.323 , -1.041 , -0.6973,  0.717 ]], dtype=float16)

## Incremental fit

In [14]:
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier

In [15]:
est = SGDClassifier(loss='log', penalty='l2', tol=1e-4)
inc = Incremental(est, scoring='accuracy')
classes = da.array([0,1])

In [18]:
t1 = time()
inc.fit(X_train, y_train, classes=classes)
t2 = time()
delt = round(t2-t1,3)
print(f"Fitting time:{delt} seconds")

c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\biaspaltsau_aa\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.wa

Fitting time:1393.92 seconds


In [19]:
t1 = time()
score=inc.score(X_test, y_test)
t2 = time()
delt = round(t2-t1,3)
print(f"Score evaluation time:{delt} seconds")
print(f"Accuracy score on the test set: {score}")

---

import pandas as pd
import numpy as np
import glob
import dask
import dask.dataframe as dd
import gc